In [104]:
# hier lokalen Dateipfad einsetzen
input_file_name = r"C:\Users\tinag\Desktop\COUNTER_Journal_Report.zip"
result_file_name = r"C:\Users\tinag\Desktop\COUNTER_Auswertung.csv"

In [105]:
import pandas as pd
#pandas importieren

from zipfile import ZipFile
# zipfile importieren, um zip-Datein nutzen zu können

import csv
# csv importieren, um csv einzulesen

import tempfile
# zur Ablage von tempfiles


from pathlib import Path
# zur Erzeugung des lokalen Verzeichnisses zur Speicherung der tempfiles

import numpy as np
# numpy Library importieren

import shutil
# shutil Library importieren

In [106]:
# lokales Verzeichnis zur Weiterarbeit überschreiben

subdir = Path(tempfile.gettempdir()).joinpath('Journal_Report_VZ')
# alte Version löschen
shutil.rmtree(subdir)
subdir.mkdir(exist_ok=True, parents=True)
# lokales Verzeichnis "Journal_Report_VZ" (= subdir) erzeugen

In [107]:
with ZipFile(input_file_name, 'r') as zip:
    zip.printdir()
    zip.extractall(subdir)
# zipfile auslesen
# Datein temporär speichern

File Name                                             Modified             Size
TR_J1_2021_ieee.csv                            2022-07-15 13:34:44        83493
TR_J1_Fake2019_ieee.csv                        2022-08-21 11:19:52        88164
TR_J1_Fake2020_ieee.csv                        2022-08-21 11:23:48        88945


In [108]:
print(subdir)
# erstelltes Verzeichnis subdir anzeigen

C:\Users\tinag\AppData\Local\Temp\Journal_Report_VZ


In [109]:
dataframes = []
headers = []
# Dataframes in leere Listen packen für spätere Bearbeitung
# Liste der geladenen Dataframes

In [110]:
# Schleife

counter_file_pattern = '*.csv'
for p in subdir.glob(counter_file_pattern):
    #Anzeige aller csv-Datein im erstellten Verzeichnis subdir
    # csv Datei bereinigen, da standardmäßiger Header mit Stammdaten im COUNTER Format bei der Weiterarbeit stört (= Zeile 0 bis 11)
    # COUNTER Daten beginnen erst in Zeile 15
    # der Stammdaten-Header (= Zeile 0 bis 11) soll als Spalten in die Tabelle integriert werden
    headers.append(pd.read_csv(str(p), sep=",", header=None)[0:12])
    
    counter_file_name_fixed = str(p).replace('csv', 'fixed.csv')
    # Kaputtes CSV von IEEE reparieren
    # In diesem CSV wird die ganze Zeile in Anführungszeichen gesetzt
    # Dies stört die Erzeugung des Dataframes, daher müssen diese entfernt werde

    df = pd.read_csv(str(p), skiprows=13)
    print(df)
    # ab Zeile 14 einlesen

    df.to_csv(counter_file_name_fixed,quoting=csv.QUOTE_NONE, escapechar=' ', index=False)
    # ohne zusätzliche Anführungszeichen abspeichern
    
    dataframes.append(pd.read_csv(counter_file_name_fixed, error_bad_lines=False))
    # fehlerhafte Zeilen ignorieren, daher folgende Fehlermeldung
    # führt zu Datenverlust

    Title,"Publisher","Publisher_ID","Platform","DOI","Proprietary_ID","Print_ISSN","Online_ISSN","URI","Metric_Type","Reporting_Period_Total","Jan-2021","Feb-2021","Mar-2021","Apr-2021","May-2021","Jun-2021","Jul-2021","Aug-2021","Sep-2021","Oct-2021","Nov-2021","Dec-2021";;
0    Bell Labs Technical Journal,"Nokia Bell Labs",...                                                                                                                                                                                                                              
1    Bell Labs Technical Journal,"Nokia Bell Labs",...                                                                                                                                                                                                                              
2    China Communications,"IEEE","","IEEE Xplore","...                                                                                                                   

C:\Users\tinag\anaconda3\anaconda3b\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 140: expected 23 fields, saw 25\nSkipping line 141: expected 23 fields, saw 25\nSkipping line 154: expected 23 fields, saw 25\nSkipping line 155: expected 23 fields, saw 25\nSkipping line 172: expected 23 fields, saw 24\nSkipping line 173: expected 23 fields, saw 24\nSkipping line 174: expected 23 fields, saw 25\nSkipping line 175: expected 23 fields, saw 25\nSkipping line 286: expected 23 fields, saw 25\nSkipping line 287: expected 23 fields, saw 25\nSkipping line 288: expected 23 fields, saw 26\nSkipping line 289: expected 23 fields, saw 26\nSkipping line 290: expected 23 fields, saw 26\nSkipping line 291: expected 23 fields, saw 26\nSkipping line 292: expected 23 fields, saw 25\nSkipping line 293: expected 23 fields, sa

In [111]:
for header_df in headers:
    print(header_df)
# Ausgabe des Stammdaten-Header aller Dataframes

                    0                                                  1
0         Report_Name             Journal Requests (Excluding OA_Gold);;
1           Report_ID                                            TR_J1;;
2             Release                                                5;;
3    Institution_Name                                THB Aschaffenburg;;
4      Institution_ID                                       ieee:96052;;
5        Metric_Types         Total_Item_Requests; Unique_Item_Requests;
6      Report_Filters  Data_Type=Journal; Access_Type=Controlled; Acc...
7   Report_Attributes                                                 ;;
8          Exceptions                                                 ;;
9    Reporting_Period        Begin_Date=2021-01-01; End_Date=2021-12-31;
10            Created                             2022-07-01T14:54:15Z;;
11         Created_By                                     MPS for IEEE;;
                    0                              

In [112]:
headers_t = []
# neue Liste für transponierte Header
for header_df in headers:
    head_t_df = header_df.T
    head_t_df.columns = head_t_df.iloc[0]
    headers_t.append(head_t_df[1:])
# transponieren

In [113]:
for head_t in headers_t:
    print(head_t)

0                             Report_Name Report_ID Release  \
1  Journal Requests (Excluding OA_Gold);;   TR_J1;;     5;;   

0     Institution_Name Institution_ID  \
1  THB Aschaffenburg;;   ieee:96052;;   

0                                Metric_Types  \
1  Total_Item_Requests; Unique_Item_Requests;   

0                                     Report_Filters Report_Attributes  \
1  Data_Type=Journal; Access_Type=Controlled; Acc...                ;;   

0 Exceptions                             Reporting_Period  \
1         ;;  Begin_Date=2021-01-01; End_Date=2021-12-31;   

0                 Created      Created_By  
1  2022-07-01T14:54:15Z;;  MPS for IEEE;;  
0                             Report_Name Report_ID Release  \
1  Journal Requests (Excluding OA_Gold);;  TR_J12;;     5;;   

0     Institution_Name Institution_ID  \
1  THB Aschaffenburg;;  ieee:960533;;   

0                                Metric_Types  \
1  Total_Item_Requests; Unique_Item_Requests;   

0                     

In [114]:
for df in dataframes:
    print(df)
# In der ersten Spalte fehlen die Anführungszeichen; aktuell ignorieren wir diese Zeilen

                                                Title           Publisher   \
0                      Bell  Labs  Technical  Journal   Nokia  Bell  Labs    
1                      Bell  Labs  Technical  Journal   Nokia  Bell  Labs    
2                               China  Communications                IEEE    
3                               China  Communications                IEEE    
4                                            Computer                IEEE    
..                                                 ...                 ...   
311                         Proceedings  of  the  IRE                IEEE    
312             The  Bell  System  Technical  Journal   Nokia  Bell  Labs    
313             The  Bell  System  Technical  Journal   Nokia  Bell  Labs    
314  Transactions  of  the  American  Institute  of...               IEEE    
315  Transactions  of  the  American  Institute  of...               IEEE    

    Publisher_ID       Platform                      DOI  Propr

In [115]:
for i in range(len(dataframes)):
    dataframes[i] = dataframes[i].loc[df['Metric_Type '] == 'Unique_Item_Requests ',:]
    print(dataframes[i])
# Ausgabe aller Zeilen, die den Wert 'Unique_Item_Requests ' (mit Leerzeichen) enthalten

                                                Title           Publisher   \
1                      Bell  Labs  Technical  Journal   Nokia  Bell  Labs    
3                               China  Communications                IEEE    
5                                            Computer                IEEE    
7              Computing  in  Science  &  Engineering                IEEE    
9                          Engineering  &  Technology                 IET    
..                                                 ...                 ...   
307      Journal  of  Microelectromechanical  Systems                IEEE    
309                        Proceedings  of  the  IEEE                IEEE    
311                         Proceedings  of  the  IRE                IEEE    
313             The  Bell  System  Technical  Journal   Nokia  Bell  Labs    
315  Transactions  of  the  American  Institute  of...               IEEE    

    Publisher_ID       Platform                      DOI  Propr

In [116]:
for i in range(len(dataframes)):
    counter_unique_item_requests = "ieee_counter_unique_item_requests" + str(i)
    dataframes[i].to_csv(counter_unique_item_requests, index=False)
    print('Zwischenergebnis nach {} gespeichert.'.format(counter_unique_item_requests))
# Zwischenergebnis als Datei abspeichern (da ggf. für weitere Auswertung benötigt)

Zwischenergebnis nach ieee_counter_unique_item_requests0 gespeichert.
Zwischenergebnis nach ieee_counter_unique_item_requests1 gespeichert.
Zwischenergebnis nach ieee_counter_unique_item_requests2 gespeichert.


In [117]:
for df in dataframes:
    print(df.columns)

Index(['Title ', 'Publisher ', 'Publisher_ID ', 'Platform ', 'DOI ',
       'Proprietary_ID ', 'Print_ISSN ', 'Online_ISSN ', 'URI ',
       'Metric_Type ', 'Reporting_Period_Total ', 'Jan-2021 ', 'Feb-2021 ',
       'Mar-2021 ', 'Apr-2021 ', 'May-2021 ', 'Jun-2021 ', 'Jul-2021 ',
       'Aug-2021 ', 'Sep-2021 ', 'Oct-2021 ', 'Nov-2021 ', 'Dec-2021;;'],
      dtype='object')
Index(['Title ', 'Publisher ', 'Publisher_ID ', 'Platform ', 'DOI ',
       'Proprietary_ID ', 'Print_ISSN ', 'Online_ISSN ', 'URI ',
       'Metric_Type ', 'Reporting_Period_Total ', 'Jan-2019 ', 'Feb-2019 ',
       'Mar-2019 ', 'Apr-2019 ', 'May-2019 ', 'Jun-2019 ', 'Jul-2019 ',
       'Aug-2019 ', 'Sep-2019 ', 'Oct-2019 ', 'Nov-2019 ', 'Dec-2019;;'],
      dtype='object')
Index(['Title ', 'Publisher ', 'Publisher_ID ', 'Platform ', 'DOI ',
       'Proprietary_ID ', 'Print_ISSN ', 'Online_ISSN ', 'URI ',
       'Metric_Type ', 'Reporting_Period_Total ', 'Jan-2020 ', 'Feb-2020 ',
       'Mar-2020 ', 'Apr-2020 ', '

In [118]:
for i in range(len(dataframes)):
    df = dataframes[i]
    for column in df.columns:
        new_column_name = column.split('-')[0].strip()
        df = df.rename(columns={column:new_column_name})
    # generische Spaltennamen
    dataframes[i] = df
    print(df.columns)

Index(['Title', 'Publisher', 'Publisher_ID', 'Platform', 'DOI',
       'Proprietary_ID', 'Print_ISSN', 'Online_ISSN', 'URI', 'Metric_Type',
       'Reporting_Period_Total', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='object')
Index(['Title', 'Publisher', 'Publisher_ID', 'Platform', 'DOI',
       'Proprietary_ID', 'Print_ISSN', 'Online_ISSN', 'URI', 'Metric_Type',
       'Reporting_Period_Total', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='object')
Index(['Title', 'Publisher', 'Publisher_ID', 'Platform', 'DOI',
       'Proprietary_ID', 'Print_ISSN', 'Online_ISSN', 'URI', 'Metric_Type',
       'Reporting_Period_Total', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
      dtype='object')


In [119]:
for df in dataframes:
    print(df[['Reporting_Period_Total',  'Jan', 'Feb',
       'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].sum())
# Summe der Spalten bilden
# Problem: in Spalte "Dec" gibt es am Ende immer ;;

Reporting_Period_Total                                                 1372
Jan                                                                     175
Feb                                                                     112
Mar                                                                      95
Apr                                                                     119
May                                                                     102
Jun                                                                     113
Jul                                                                      74
Aug                                                                     101
Sep                                                                     117
Oct                                                                      93
Nov                                                                     186
Dec                       0;;0;;0;;0;;0;;8;;0;;0;;0;;0;;0;;1;;0;;0;;1;;1...
dtype: objec

In [120]:
for df in dataframes:
    df[['Dec', 'Rest01','Rest02']]=df.Dec.str.split(';',expand=True)
    print(df)
# Aufteilung der Inhalte aus der Spalte Dec in 3 Spalten

                                                 Title           Publisher  \
1                      Bell  Labs  Technical  Journal   Nokia  Bell  Labs    
3                               China  Communications                IEEE    
5                                            Computer                IEEE    
7              Computing  in  Science  &  Engineering                IEEE    
9                          Engineering  &  Technology                 IET    
..                                                 ...                 ...   
307      Journal  of  Microelectromechanical  Systems                IEEE    
309                        Proceedings  of  the  IEEE                IEEE    
311                         Proceedings  of  the  IRE                IEEE    
313             The  Bell  System  Technical  Journal   Nokia  Bell  Labs    
315  Transactions  of  the  American  Institute  of...               IEEE    

    Publisher_ID       Platform                      DOI Propri

In [121]:
for df in dataframes:
    print(df[['Reporting_Period_Total',  'Jan', 'Feb',
       'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].sum())
    # Summenbildung schlägt fehlt, weil Datatype nicht überall int64

Reporting_Period_Total                                                 1372
Jan                                                                     175
Feb                                                                     112
Mar                                                                      95
Apr                                                                     119
May                                                                     102
Jun                                                                     113
Jul                                                                      74
Aug                                                                     101
Sep                                                                     117
Oct                                                                      93
Nov                                                                     186
Dec                       0000080000010011000000000011000000100100000010...
dtype: objec

In [122]:
for df in dataframes:
    print(df.dtypes)
    #Anzeige Datatypes

Title                     object
Publisher                 object
Publisher_ID              object
Platform                  object
DOI                       object
Proprietary_ID            object
Print_ISSN                object
Online_ISSN               object
URI                       object
Metric_Type               object
Reporting_Period_Total     int64
Jan                        int64
Feb                        int64
Mar                        int64
Apr                        int64
May                        int64
Jun                        int64
Jul                        int64
Aug                        int64
Sep                        int64
Oct                        int64
Nov                        int64
Dec                       object
Rest01                    object
Rest02                    object
dtype: object
Title                     object
Publisher                 object
Publisher_ID              object
Platform                  object
DOI                       obj

In [123]:
for df in dataframes:
    df["Dec"] = df["Dec"].astype(str).astype(int)
    print(df.dtypes)
#Umwandlung datatype object in int für relevante Spalten
#Anzeige Datatypes

Title                     object
Publisher                 object
Publisher_ID              object
Platform                  object
DOI                       object
Proprietary_ID            object
Print_ISSN                object
Online_ISSN               object
URI                       object
Metric_Type               object
Reporting_Period_Total     int64
Jan                        int64
Feb                        int64
Mar                        int64
Apr                        int64
May                        int64
Jun                        int64
Jul                        int64
Aug                        int64
Sep                        int64
Oct                        int64
Nov                        int64
Dec                        int32
Rest01                    object
Rest02                    object
dtype: object
Title                     object
Publisher                 object
Publisher_ID              object
Platform                  object
DOI                       obj

In [124]:
sums = []
# neue Liste für Summen

for df in dataframes:
    sum_df = df[['Reporting_Period_Total',  'Jan', 'Feb',
       'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec']].sum().to_frame().T
    # Summe berechnen Umwandlung in Dataframe
    sums.append(sum_df)
    print(sum_df)

   Reporting_Period_Total  Jan  Feb  Mar  Apr  May  Jun  Jul  Aug  Sep  Oct  \
0                    1372  175  112   95  119  102  113   74  101  117   93   

   Nov  Dec  
0  186   85  
   Reporting_Period_Total  Jan  Feb   Mar   Apr  May   Jun  Jul  Aug   Sep  \
0                   43644  834  609  1961  1987  667  4154  681  844  1064   

    Oct   Nov  Dec  
0  1925  1147  766  
   Reporting_Period_Total  Jan  Feb   Mar   Apr  May   Jun  Jul   Aug   Sep  \
0                   95819  771  587  2396  1859  637  4109  654  1090  1014   

    Oct   Nov   Dec  
0  1784  1107  1013  


In [125]:
results = []
# neue Liste für Zusammenführung der Dataframes

for i in range(len(sums)):
    head_t_df = headers_t[i].assign(id=0)
    sum_df = sums[i].assign(id=0)
    rdf = head_t_df.join(sum_df, on="id", lsuffix='x', rsuffix='y')
    # Dataframe "Header" und Dataframe "Summe" zusammenführen
    print(rdf)
    results.append(rdf)

                              Report_Name Report_ID Release  \
1  Journal Requests (Excluding OA_Gold);;   TR_J1;;     5;;   

      Institution_Name Institution_ID  \
1  THB Aschaffenburg;;   ieee:96052;;   

                                 Metric_Types  \
1  Total_Item_Requests; Unique_Item_Requests;   

                                      Report_Filters Report_Attributes  \
1  Data_Type=Journal; Access_Type=Controlled; Acc...                ;;   

  Exceptions                             Reporting_Period  ...  Apr  May  Jun  \
1         ;;  Begin_Date=2021-01-01; End_Date=2021-12-31;  ...  119  102  113   

   Jul  Aug  Sep  Oct  Nov  Dec  idy  
1   74  101  117   93  186   85    0  

[1 rows x 27 columns]
                              Report_Name Report_ID Release  \
1  Journal Requests (Excluding OA_Gold);;  TR_J12;;     5;;   

      Institution_Name Institution_ID  \
1  THB Aschaffenburg;;  ieee:960533;;   

                                 Metric_Types  \
1  Total_Item_Reque

In [126]:
result_df = pd.concat(results)
# Einzelergebnisse zu einem einzigen Dataframe zusammenführen
result_df

,Report_Name,Report_ID,Release,Institution_Name,Institution_ID,Metric_Types,Report_Filters,Report_Attributes,Exceptions,Reporting_Period,...,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,idy
1,Journal Requests (Excluding OA_Gold);;,TR_J1;;,5;;,THB Aschaffenburg;;,ieee:96052;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;,Begin_Date=2021-01-01; End_Date=2021-12-31;,...,119,102,113,74,101,117,93,186,85,0
1,Journal Requests (Excluding OA_Gold);;,TR_J12;;,5;;,THB Aschaffenburg;;,ieee:960533;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;,Begin_Date=Fake_2019; End_Date=2019-1233-312;,...,1987,667,4154,681,844,1064,1925,1147,766,0
1,Journal Requests (Excluding OA_Gold);;,TR_J11;;,5;;,THB Aschaffenburg;;,ieee:960633;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;,Begin_Date=Fake_2020; End_Date=2020-1133-311;,...,1859,637,4109,654,1090,1014,1784,1107,1013,0


In [127]:
result_df.columns

Index(['Report_Name', 'Report_ID', 'Release', 'Institution_Name',
       'Institution_ID', 'Metric_Types', 'Report_Filters', 'Report_Attributes',
       'Exceptions', 'Reporting_Period', 'Created', 'Created_By', 'idx',
       'Reporting_Period_Total', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'idy'],
      dtype='object')

In [128]:
sort_result_df = result_df[['Report_Name', 'Created_By', 'Reporting_Period_Total', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'Jul', 'Aug',
       'Sep', 'Oct', 'Nov', 'Dec', 'Reporting_Period', 'Created','Report_ID', 'Release', 'Institution_Name',
       'Institution_ID', 'Metric_Types', 'Report_Filters', 'Report_Attributes',
       'Exceptions',]]
sort_result_df
#Spalten neu ordnen


,Report_Name,Created_By,Reporting_Period_Total,Jan,Feb,Mar,Apr,May,Jun,Jul,...,Reporting_Period,Created,Report_ID,Release,Institution_Name,Institution_ID,Metric_Types,Report_Filters,Report_Attributes,Exceptions
1,Journal Requests (Excluding OA_Gold);;,MPS for IEEE;;,1372,175,112,95,119,102,113,74,...,Begin_Date=2021-01-01; End_Date=2021-12-31;,2022-07-01T14:54:15Z;;,TR_J1;;,5;;,THB Aschaffenburg;;,ieee:96052;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;
1,Journal Requests (Excluding OA_Gold);;,MPS for IEEE;;,43644,834,609,1961,1987,667,4154,681,...,Begin_Date=Fake_2019; End_Date=2019-1233-312;,2019-07-012T124:54:125Z;;,TR_J12;;,5;;,THB Aschaffenburg;;,ieee:960533;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;
1,Journal Requests (Excluding OA_Gold);;,MPS for IEEE;;,95819,771,587,2396,1859,637,4109,654,...,Begin_Date=Fake_2020; End_Date=2020-1133-311;,2020-06-011T112:62:116Z;;,TR_J11;;,5;;,THB Aschaffenburg;;,ieee:960633;;,Total_Item_Requests; Unique_Item_Requests;,Data_Type=Journal; Access_Type=Controlled; Acc...,;;,;;


In [129]:
sort_result_df.to_csv(result_file_name,index=False)

# Uwandlung DataFrame "Ergebniss_Journal_Report" in csv-Dateiformat
# erstellt Datei im oben angegeben Verzeichnis

print('Auswertungsdatei wurde erfolgreich nach {} geschrieben.'.format(result_file_name))
# Datei kann nicht direkt in Excel geöffnet werden
# muss über die Funktion "Daten abrufen" in Excel geladen werden

Auswertungsdatei wurde erfolgreich nach C:\Users\tinag\Desktop\COUNTER_Auswertung.csv geschrieben.
